### Machine-to-Machine (M2M) API:

---

#### Pros:

1. **Same for Manual Download**: The same API we used for downloading data manually can be used for automating the process.

2. **Easy to Use**: The API is easy to use and doesn't require any special setup. Only USGS account and password are required.

---

#### Cons:

1. **Learning Curve**: Need to get familiar with the API and its parameters. e.g. how to find the metal text and the corresponding band files.

2. **Little Technical Support**: The API is not well documented and there is little technical support. Website: "This service is provided "as is" without guarantee of technical assistance."
---

#### How-to:

**Run block by block and remember to change login data**

In [1]:
# import libraries
import requests
import json
import pandas as pd

**Utility Fucntion to Connect to the M2M API**

In [2]:
# utility function to post request to M2M API
def post_request(endpoint, data, auth_token = None):
    login_url = 'https://m2m.cr.usgs.gov/api/api/json/stable/' + endpoint
    headers = {
        'X-Auth-Token': auth_token
    }
    response = requests.post(login_url, json=data, headers=headers)
    if response.status_code != 200:
        print("Error: " + str(response.status_code))
        return None
    response = response.json()
    response = json.dumps(response, indent=4)
    return response

**Config:**

In [4]:
# the following is the login data for the M2M API
# notice the TODOs, you need to replace them with your own username and password
login_data = {
    "username": "tongjin",
    "password": "Link9jintong"
}

# the identifier for the dataset we will be using
dataset_id = "5e81f14f59432a27"
dataset_name = "landsat_ot_c2_l1"

# the csv file containing the list of scenes we want to download
csv_file = "WRS-2_reduced_regions_coordinates\WRS-2_R56_R60_Regions\R56_R60_P6_P10_coordinates.csv"

# default scene search json
# program will update the spatial filter's coordinates
# you can change other parameters if you want, such as cloud cover and time range
scene_data ={
    "maxResults": 10000,
    "datasetName": dataset_name,
    "sceneFilter": {
        "spatialFilter": {
            "filterType": "mbr",
            "lowerLeft": {
                "latitude": 0,
                "longitude":  0
            },
            "upperRight": {
                    "latitude": 0,
                    "longitude": 0
            }
        },
        "metadataFilter": None,
        "cloudCoverFilter": {
            "max": 10,
            "min": 0,
            "includeUnknown": False
        },
        "acquisitionFilter": {
            "end": "2023-11-06",
            "start": "2022-01-01"
        }
    },
    "metadataType": "summary",
}

# the number of images we want to download for each scene
num_images_per_scene = 5

# Login to get the API key 
# Notice the api key is valid for only a few hours
# Need to rerun this cell if the api key is expired
api_key = json.loads(post_request("login", login_data))['data']

In [5]:
# read in the csv file for the bounding box
def get_coordinates_from_csv(input_csv_file):
    # read in csv file for the bounding box
    coordinates = pd.read_csv(input_csv_file)

    # initialize the output coordinates
    output_coordinates = []

    # iterate through the csv file to get the coordinates
    for index, row in coordinates.iterrows():
        output_coordinate_i = {}
        output_coordinate_i['LL Lat'] = row['LL Lat']
        output_coordinate_i['LL Lon'] = row['LL Lon']
        output_coordinate_i['UR Lat'] = row['UR Lat']
        output_coordinate_i['UR Lon'] = row['UR Lon']
        output_coordinates.append(output_coordinate_i) 

    return output_coordinates

In [6]:
# given a list of coordinates, get the scenes
def get_scenes(coordinates):
    # iterate through the coordinates to get the scenes
    global scene_data
    global api_key
    scene_search_responses = []
    for coordinate_i in coordinates:
        scene_data["sceneFilter"]["spatialFilter"]["lowerLeft"]["latitude"] = coordinate_i['LL Lat']
        scene_data["sceneFilter"]["spatialFilter"]["lowerLeft"]["longitude"] = coordinate_i['LL Lon']
        scene_data["sceneFilter"]["spatialFilter"]["upperRight"]["latitude"] = coordinate_i['UR Lat']
        scene_data["sceneFilter"]["spatialFilter"]["upperRight"]["longitude"] = coordinate_i['UR Lon']
        scene_search_response = post_request("scene-search",scene_data,api_key)
        # print("The length of the result is "+ str(json.loads(scene_search_response)['data']['recordsReturned']))
        scene_search_responses.append(scene_search_response)
    return scene_search_responses

In [7]:
# given a list of scenes, get the images
def get_images_id_from_scenes(scenes):
    global num_images_per_scene
    top_resutls_list = {}
    for scene_i in scenes:
        results_list = json.loads(scene_i)['data']['results']
        results_list_len = min(num_images_per_scene,len(results_list))
        for i in range(results_list_len):
            data_entity_id = results_list[i]['entityId']
            download_options = post_request("download-options",{"datasetName":dataset_name,"entityIds":[data_entity_id]},api_key)
            download_options = json.loads(download_options)
            top_resutls_list[data_entity_id] = []
            for image_i in download_options['data'][0]["secondaryDownloads"]:
                if image_i['entityId'].endswith('B2_TIF') or image_i['entityId'].endswith('B3_TIF') or image_i['entityId'].endswith('B4_TIF') or image_i['entityId'].endswith('MTL_TXT'):
                    top_resutls_list[data_entity_id].append({
                        "entityId": image_i['entityId'],
                        "productId":image_i['id']
                    })
            if len(top_resutls_list[data_entity_id]) != 4:
                print("Error: the length of the top_resutls_list[data_entity_id] is not 4")
        # download_options['data'] will return several download sources, we just pick the first one as [0]
    return top_resutls_list

In [8]:
# get the download links for the images
# notice the links may not be valid instantly, you may need to wait for several seconds
def get_download_links(top_resutls_lists):
    # iterate through the top_resutls_lists dictionary to download the images
    download_links = {}
    for scene_id, result_list_i in top_resutls_lists.items():
        download_args ={"downloads":result_list_i}
        download = post_request("download-request",download_args, api_key)
        download_links[scene_id] = download
    return download_links

In [25]:
#  get the download data for the images
def get_download_data(download_links):
    for scene_id, download_link_i in download_links.items():
        # Parse the JSON data
        download_data = json.loads(download_link_i)

        # iterate the available downloads
        for download_data_i in download_data['data']['availableDownloads']:
            # Extract the download URL for the first file
            download_url = download_data_i['url']

            # Save the file to your laptop
            response = requests.get(download_url)
            if response.status_code == 200:
                with open(download_url.split('?')[0].split('/')[-1], 'wb') as file:
                    file.write(response.content)
                # print("File saved to 'test.TIF'")
            else:
                print("Failed to download the file.")

# Complete Pipeline to use the functions above

In [10]:
output_coordinates = get_coordinates_from_csv(csv_file)
print(output_coordinates)
print("----------------------")
scene_search_responses = get_scenes(output_coordinates)
print(scene_search_responses)
print("----------------------")
images_for_each_scene = get_images_id_from_scenes(scene_search_responses)
print(images_for_each_scene)
print("----------------------")

[{'LL Lat': -0.685, 'LL Lon': -81.071, 'UR Lat': 0.685, 'UR Lon': -79.031}, {'LL Lat': 0.762, 'LL Lon': -80.764, 'UR Lat': 2.131, 'UR Lon': -78.723}, {'LL Lat': 5.101, 'LL Lon': -78.299, 'UR Lat': 6.468, 'UR Lon': -76.248}, {'LL Lat': 3.654, 'LL Lon': -78.606, 'UR Lat': 5.023, 'UR Lon': -76.56}, {'LL Lat': 2.208, 'LL Lon': -78.912, 'UR Lat': 3.577, 'UR Lon': -76.869}, {'LL Lat': 0.762, 'LL Lon': -79.219, 'UR Lat': 2.131, 'UR Lon': -77.178}]
----------------------
['{\n    "requestId": 1522684742,\n    "version": "stable",\n    "sessionId": 253652142,\n    "data": {\n        "results": [\n            {\n                "browse": [\n                    {\n                        "id": "5e81f7d39594374",\n                        "browseRotationEnabled": null,\n                        "browseName": "Reflective Browse",\n                        "browsePath": "https://landsatlook.usgs.gov/gen-browse?size=rrb&type=refl&product_id=LC09_L1TP_011059_20230328_20230329_02_T1",\n                   

In [11]:
# You may need to run this cell several times to be sure that the download links are valid
# Q: How to check if the download links are valid?
# Ans: They are valid if the download links are under "availableDownloads" in the response
download_links = get_download_links(images_for_each_scene)
print(json.dumps(download_links, indent=4))

{
    "LC90110592023087LGN00": "{\n    \"requestId\": 1522733535,\n    \"version\": \"stable\",\n    \"sessionId\": 253652142,\n    \"data\": {\n        \"availableDownloads\": [\n            {\n                \"downloadId\": 500513411,\n                \"eulaCode\": null,\n                \"url\": \"https://landsatlook.usgs.gov/data/collection02/level-1/standard/oli-tirs/2023/011/059/LC09_L1TP_011059_20230328_20230329_02_T1/LC09_L1TP_011059_20230328_20230329_02_T1_B2.TIF?requestSignature=eyJjb250YWN0SWQiOjI2OTQxNDg3LCJkb3dubG9hZElkIjo1MDA1MTM0MTEsImRhdGVHZW5lcmF0ZWQiOiIyMDIzLTExLTE4VDE3OjQ1OjU5LTA2OjAwIiwiaWQiOiJMQzA5X0wxVFBfMDExMDU5XzIwMjMwMzI4XzIwMjMwMzI5XzAyX1QxX0IyLlRJRiIsInNpZ25hdHVyZSI6IiQ1JCRYQWZlbE5zaFZWcC5EUDB5Wk04THZtVnVwckVWS0lLalpDNFpUbG1QVUxDIn0=\"\n            },\n            {\n                \"downloadId\": 500513412,\n                \"eulaCode\": null,\n                \"url\": \"https://landsatlook.usgs.gov/data/collection02/level-1/standard/oli-tirs/2023/011/059/

In [26]:
# this will download the images to your local machine
# we download the first scene for example here
download_links = {"LC90110592023087LGN00" : download_links["LC90110592023087LGN00"]}

get_download_data(download_links)

# Notice the metaldata is also downloaded as TIF file on your local machine
# This is because we can't tell the difference between the metadata and the image given the download link M2M API provides

# Other not used API Calls:

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.image import imread

# Assuming you have saved the .tif file as 'downloaded_file.tif'
image_path = 'test.tif'

# Load and display the image
image = imread(image_path)
plt.imshow(image, cmap='gray')  # 'cmap' specifies the color map, you can change it as needed
plt.axis('off')  # Turn off axis labels
plt.title('TIF Image')
plt.show()


In [ ]:
dataset_search_response = post_request("dataset",{"datasetName":dataset_name},api_key)
print(dataset_search_response)

In [ ]:
dataset_download_options_args ={
    "datasetName": dataset_name,
    "sceneFilter": {
        "spatialFilter": {
            "filterType": "mbr",
            "lowerLeft": {
                "latitude": 40,
                "longitude":  -79
            },
            "upperRight": {
                    "latitude": 50,
                    "longitude": -80
            }
        },
        "metadataFilter": None,
        "cloudCoverFilter": {
            "max": 100,
            "min": 0,
            "includeUnknown": True
        },
    },
}
dataset_download_options = post_request("dataset-download-options",dataset_download_options_args,api_key)
print(dataset_download_options)